In [1]:
from __future__ import print_function
import freesound
import random
import utils
import json
from IPython.core.display import display, HTML
from collections import defaultdict

API_KEY='Ba3FmvMrgAuJSOT2MmaU6LC9BzjlwkXq49AZ5ib1'
c = freesound.FreesoundClient()
c.set_token(API_KEY,"token")
PAGE_SIZE = 50 # Page size for fs requests, no need to change that

In [2]:
# Configure dataset parameters and audio categories

DATASET_NAME = '../../json_final/generated/baseline' # Dataset will be saved in a .json file with this name
N = None # Number of sounds per class (set 'None' to get all in dataset)
DATASET_CLASSES = ["Bathtub (filling or washing)",
  "Chopping (food)",
  "Blender",
  "Coin (dropping)",
  "Cupboard open or close",
  "Cutlery,silverware",
  "Dishes, pots, and pans",
  "Door",
  "Drawer open or close",
  "Electric shaver, electric razor",
  "Frying (food)",
  "Hair dryer",
  "Kettle whistle",
  "Keys jangling",
  "Microwave oven",
  "Packing tape, duct tape",
  "Scissors",
  "Shuffling cards",
  "Sink (filling or washing)",
  "Toilet flush",
  "Toothbrush",
  "Typing",
  "Vacuum cleaner",
  "Velcro, hook and loop fastener",
  "Water tap, faucet",
  "Writing",
  "Zipper (clothing)"
]
# Get sound examples from Freesound
data_index = json.load(open('../../json_final/generated/dataset_categories_more_than_90.json'))
dataset = defaultdict(list)

In [3]:
for klass in DATASET_CLASSES:
    sound_ids = data_index.get(klass, None)
    # retrieve all the ids of the current category
    if sound_ids is None:
        print('Skipping class %s as no data is available for it' % klass)
        continue
    # if there are any skip to the next category(it depends from the data_index.json)
    random.shuffle(sound_ids)  # Shuffle order of sound ids
    sound_ids = sound_ids[:N]  # Limit number of selected sound ids

    print('Getting sounds\' data for class %s...' % klass)
    for i in range(0, len(sound_ids), PAGE_SIZE):
        current_sound_ids = sound_ids[i:i + PAGE_SIZE]

        fields = "id,name,description,analysis"
        descriptors = "sfx.duration,sfx.logattacktime,sfx.strongdecay,sfx.inharmonicity,lowlevel.silence_rate_20dB," \
                      "lowlevel.mfcc,lowlevel.zerocrossingrate,lowlevel.spectralflux, " \
                      "lowlevel.silence_rate_30dB, lowlevel.silence_rate_60dB, lowlevel.spectral_energyband_low," \
                      "lowlevel.spectral_energyband_middle_low, lowlevel.spectral_energyband_middle_high," \
                      "lowlevel.spectral_energyband_high"
        results_pager = c.text_search(
            filter='id:(%s)' % ' OR '.join([str(sid) for sid in current_sound_ids]),
            page_size=PAGE_SIZE,
            fields=fields,
            descriptors=descriptors
        )
        dataset[klass] += results_pager.results

        # TIP ON AUDIO FEATURES: you can get also audio features extracted in freesound by passing a 'descriptors'
        # parameter in the text_search function and including 'analysis' in the fields list
        # (see http://www.freesound.org/docs/api/resources_apiv2.html#response-sound-list):
        #
        # fields = "id,tags,description,username,analysis"
        # descriptors = "lowlevel.spectral_centroid,lowlevel.barkbands.mean"
        #
        # e.g.: results_page = c.text_search(query=target_query, ..., fields=fields, descriptors=descriptors)
        # ...

Skipping class Bathtub (filling or washing) as no data is available for it
Skipping class Chopping (food) as no data is available for it
Skipping class Blender as no data is available for it
Getting sounds' data for class Coin (dropping)...
Getting sounds' data for class Cupboard open or close...
Skipping class Cutlery,silverware as no data is available for it
Skipping class Dishes, pots, and pans as no data is available for it
Getting sounds' data for class Door...
Getting sounds' data for class Drawer open or close...
Skipping class Electric shaver, electric razor as no data is available for it
Skipping class Frying (food) as no data is available for it
Skipping class Hair dryer as no data is available for it
Skipping class Kettle whistle as no data is available for it
Getting sounds' data for class Keys jangling...
Getting sounds' data for class Microwave oven...
Getting sounds' data for class Packing tape, duct tape...
Getting sounds' data for class Scissors...
Skipping class Shuff

In [4]:
# Show information and save dataset to file so we can work with it later on
utils.save_to_json('%s.json' % DATASET_NAME, dataset)
print('\nDataset created with %i classes:' % len(dataset))
for klass, sounds in dataset.items():
    print('\t%s: %i sounds' % (klass, len(sounds)))
print('Saved to %s' % '%s.json' % DATASET_NAME)


Dataset created with 13 classes:
	Coin (dropping): 127 sounds
	Cupboard open or close: 104 sounds
	Door: 155 sounds
	Drawer open or close: 117 sounds
	Keys jangling: 122 sounds
	Microwave oven: 98 sounds
	Packing tape, duct tape: 95 sounds
	Scissors: 126 sounds
	Toilet flush: 156 sounds
	Typing: 153 sounds
	Vacuum cleaner: 102 sounds
	Writing: 147 sounds
	Zipper (clothing): 132 sounds
Saved to ../../json_final/generated/baseline.json
